## doc2vec

testing how doc2vec works


In [33]:
import pandas as pd
import scipy
import numpy as np
import pandas as pd
import matplotlib as mpl
from sklearn.model_selection import train_test_split
import copy

import gensim
#if it brings error, please check whether smart-open package is in the latest version


In [ ]:
data = pd.read_csv("data/train-balanced-sarcasm.csv")


In [35]:
#check the nan value in "comment"
s = data["comment"].isnull()

data_nan_list = s.index[s]
data_nan_list

Int64Index([  56269,   68590,  135348,  199910,  258718,  284331,  312969,
             328775,  331735,  332600,  332631,  362293,  389792,  445204,
             505371,  520619,  524263,  529336,  532823,  569280,  645450,
             651242,  661519,  675235,  683899,  747602,  799033,  800812,
             813274,  817886,  859333,  875251,  878050,  898863,  905291,
             914178,  914615,  918700,  919882,  923678,  936221,  949593,
             966886,  967116,  978220,  982492,  992907,  995023, 1001185,
            1001891, 1002133, 1009303, 1010599],
           dtype='int64')

In [36]:
data.dropna(subset=['comment', 'label'], inplace=True)
data

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...
...,...,...,...,...,...,...,...,...,...,...
1010821,1,I'm sure that Iran and N. Korea have the techn...,TwarkMain,reddit.com,2,2,0,2009-04,2009-04-25 00:47:52,"No one is calling this an engineered pathogen,..."
1010822,1,"whatever you do, don't vote green!",BCHarvey,climate,1,1,0,2009-05,2009-05-14 22:27:40,In a move typical of their recent do-nothing a...
1010823,1,Perhaps this is an atheist conspiracy to make ...,rebelcommander,atheism,1,1,0,2009-01,2009-01-11 00:22:57,Screw the Disabled--I've got to get to Church ...
1010824,1,The Slavs got their own country - it is called...,catsi,worldnews,1,1,0,2009-01,2009-01-23 21:12:49,I've always been unsettled by that. I hear a l...


In [37]:
#X_train, X_test, y_train, y_test = train_test_split(data["comment"],data["label"], test_size=0.2)

In [38]:
# function to preprocess and tokenize text
def tokenize_corpus(df, tokens_only=False):
    
    for i in df.index:
        line = df[i]
        tokens = gensim.utils.simple_preprocess(line)
        if tokens_only:
            yield tokens
        else:
            yield gensim.models.doc2vec.TaggedDocument(tokens, [i])
        #print(f"{i}")

In [39]:
def doc2vec_train(text_series, vec_size = 50, n_iterate = 40):
    # tokenize a training corpus
    corpus = list(tokenize_corpus(text_series))
    
    doc = list(tokenize_corpus(text_series, tokens_only=True))
    
    # train doc2vec on the training corpus
    model = gensim.models.doc2vec.Doc2Vec(vector_size=vec_size, min_count=2, epochs=n_iterate)
    model.build_vocab(corpus)
    model.train(corpus, total_examples=model.corpus_count, epochs=model.epochs)
    
    return model

In [40]:
def doc2vec_generate(text_series, model):
    
    doc = list(tokenize_corpus(text_series, tokens_only=True))
    
    # generate embeddings for the new documents
    doc_emb = np.zeros((len(doc),vec_size))
    for i in range(len(doc)):
        doc_emb[i,:] = model.infer_vector(doc[i])
        
    doc_emb_df = pd.DataFrame(doc_emb)
    
    return doc_emb_df

In [ ]:
# size of the output vector
n_vec = 64

# run doc2vec
model =  doc2vec_train(data["comment"], vec_size = n_vec)
doc_emb_df = doc2vec_generate(data["comment"], model)

#export it as csv
#it takes around 1.21 GB disk storage 
doc_emb_df.to_csv(f'doc_emb_{vec_size}.csv', index=False)


In [42]:
doc_emb_df

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0.127862,-0.019599,-0.087014,0.087399,0.146308,0.024058,-0.395626,0.172591,0.048516,-0.156912,...,0.105660,-0.079498,-0.331257,-0.072858,-0.050527,-0.011040,0.003591,-0.029970,-0.278884,0.126967
1,0.887663,-0.043182,0.844233,0.223901,-0.399206,0.588739,0.373345,-0.118279,-0.425396,-0.919981,...,-0.154893,-0.203879,0.001327,-0.581525,0.862273,0.105466,-0.964293,0.063333,0.610815,-0.007280
2,0.856357,-1.035831,-1.427065,0.599905,0.075403,0.151369,-0.851479,0.213169,0.551703,-1.145140,...,-0.058335,-0.398421,-0.093258,0.171745,-0.373348,-0.079395,0.344274,-0.903403,-0.030337,-0.082255
3,0.783971,0.446385,-1.095629,0.521688,-0.171933,0.262818,-0.664950,-0.458009,0.232369,0.563871,...,-0.501949,-0.141537,-0.743462,-0.635561,0.346453,0.280692,-0.179629,-0.297779,1.052319,-1.061878
4,0.341725,0.172049,0.237707,0.382002,0.052976,0.248339,-0.062441,-0.028663,0.253968,0.355791,...,-0.048106,-0.120476,-0.327721,0.222168,0.167849,0.284008,0.576967,0.296323,0.025599,-0.247098
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1010768,-0.171096,0.570544,0.220524,0.869572,-0.513877,-0.330733,0.923171,-0.181707,0.035599,-1.001252,...,0.458512,-0.551791,-0.277146,-0.517992,-0.369119,0.820267,-0.404313,-0.952601,-0.187914,-0.535368
1010769,-0.669153,0.442053,0.163949,-0.477443,-0.447482,0.406488,0.357317,0.339491,-0.291090,0.495786,...,0.383413,0.598937,-0.556525,-0.002050,-0.322856,-0.262325,-0.322588,0.430093,-0.537915,0.259395
1010770,0.143740,0.124314,0.026415,1.027849,-0.908955,-0.312088,0.865575,0.547618,0.279874,0.142160,...,0.348905,-0.428212,-0.524242,-1.015198,-0.149851,0.632178,-0.942817,0.487685,-0.089499,0.545360
1010771,-0.599016,-0.466122,0.084492,0.362513,-0.036460,-0.109052,-0.538425,0.034474,0.218019,-0.285374,...,-0.360436,0.067529,-0.326157,-0.109142,0.421554,0.331801,-0.240881,0.052833,-0.705633,0.490452
